In [17]:
import pandas as pd
import pyodbc

# Replace these with your actual credentials and server details
server = 'HAPPY-PC\SQLEXPRESS'
database = 'headphones'
username = 'victor'
password = 'super'
csv_file = 'Measurements/BryAudioReviews.csv'
table_name = 'BryAudioReviews'

# Connect to the MS SQL server
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Read the CSV file using pandas
df = pd.read_csv(csv_file)

# Create the table schema
create_table_sql = f"CREATE TABLE {table_name} ("

for col_name, col_type in zip(df.columns, df.dtypes):
    sql_type = 'NVARCHAR(MAX)'

    if col_type == 'int64':
        sql_type = 'INT'
    elif col_type == 'float64':
        sql_type = 'FLOAT'

    create_table_sql += f"[{col_name}] {sql_type},"

create_table_sql = create_table_sql.rstrip(',') + ')'

# Execute the statement to create the table
cursor.execute(create_table_sql)
conn.commit()

# Generate the INSERT statement
insert_sql = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({', '.join(['?' for _ in df.columns])})"

# Insert each row from the DataFrame into the table
for _, row in df.iterrows():
    cursor.execute(insert_sql, tuple(row))

conn.commit()

# Close the database connection
cursor.close()
conn.close()
